In [1]:
%load_ext autotime

In [2]:
from etltools import s3
from lambda_client import (
    CalculatorClient,
    run_batch_on_schedule,
)
from lambda_client.config_info import ConfigInfo

aws_info = {
    'profile_name': 'sandbox',
}

time: 665 ms


In [3]:
configs = ConfigInfo('lambda_client/lambda.cfg')

# 06: CA
# 36: NY
print configs.all_states

states = ['01', '04', '05', '06', '36']

['01', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56', '72']
time: 2.6 ms


In [4]:
uids = s3.read_json('s3n://picwell.sandbox.medicare/samples/philadelphia-2015')

print '{} uids read'.format(len(uids))

1352473 uids read
time: 5.86 s


In [5]:
client = CalculatorClient(aws_info)

# This only runs for large enough Lambda, e.g. 512 MB, and fails for 256 MB Lambda. 
# Memory determines how fast one can run.
response = client.run_batch(uids[:20], months=['01'], states=states)

print response

[[u'31970436701', 5], [u'1784327003', 5], [u'204451602', 5], [u'3154052701', 5], [u'1674435601', 5], [u'246540301', 5], [u'595989402', 5], [u'617921001', 5], [u'3158762201', 5], [u'31917048601', 5], [u'3170818001', 5], [u'894536502', 5], [u'30498125901', 5], [u'3182854701', 5], [u'1789308302', 5], [u'946101001', 5], [u'2225890201', 5], [u'3173810801', 5], [u'986282601', 5], [u'763956001', 5]]
time: 26.5 s


In [6]:
# Not sure why an adjustment factor is needed:
factor = 6

responses = run_batch_on_schedule(lambda uids: client.run_batch(uids, months=['01'], states=states),
                                  uids[:10000], num_writes_per_uid=5, mean_runtime=30, 
                                  min_writes=100*factor, max_writes=10000*factor, verbose=True)

5e-06: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9996 remaining)
1.007076: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9992 remaining)
2.013462: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9988 remaining)
3.020665: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9984 remaining)
4.025572: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9980 remaining)
5.033973: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9976 remaining)
6.042613: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9972 remaining)
7.048542: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9968 remaining)
8.055866: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9964 remaining)
9.062157: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9960 remaining)
10.06649: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9956 remaining)
11.072882: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9952 remaining)
12.080418: issuing 4 UIDs for every 1.0 seconds (0 intervals, 9948 remaining)


In [7]:
print 'number of responses: {}'.format(len(responses))

writes = 0
for response in responses:
    writes += response[1]
print 'number of writes: {}'.format(writes)

number of responses: 10000
number of writes: 50000
time: 5.69 ms
